# <div align="center" style="font-size: 50px;">Machine Intelligence Exam Part II</div>

### **Instructions**:

- Students are allowed to use any notes and material they want (physical or electronic).
- Students are not allowed to use any online resource.
- Students are not allowed to use any generative AI tool or any automatic code generation.
- Students are allowed to use any IDE they want (e.g., Visual Code, PyCharm).
- The IDE must not have any generative AI tool activated for code generation.
- Recommended python version is Python 3.
- You need to install, at least, the following packages: ``numpy``, ``matplotlib``, ``pandas``, ``jupyter`` , ``tensorflow``and ``keras``.


### **Submission**:

- The submission must be a single Python notebook.
- The notebook must be named *MI_Exam.ipynb*.
- **The notebook must be submitted in Digital Exam before the end of the exam**


# Imports and Data Loading

This code imports essential Python libraries for data manipulation, numerical computing, and visualization, and sets NumPy array print precision to two decimal places for cleaner output.

In [ ]:
import copy, math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
np.set_printoptions(precision=2)  # reduced display precision on numpy arrays


This code defines a function `load_data()` that loads a bike sharing dataset from a CSV file, processes it by removing certain columns, and returns the feature matrix X, target values y, feature names, and index values in a format suitable for machine learning tasks. If loading from the Google Drive URL fails, the function will automatically attempt to load the data from a local file "./Train.csv" which should be placed in the same directory as the notebook.

-----------------------------------------------------

Look at the code of the method `load_data()` to know in which folder you have to place the given file `Data.csv` containing the dataset. Alternatively, modify the code of the method to properly read the file `Data.csv`.

-----------------------------------------------------


In [ ]:
def load_data():
    try:
        # First try loading from URL
        df = pd.read_csv("https://drive.google.com/uc?id=1AeJXQa_BpRrUwUbzmQlqzFhMafOnuWwx", sep=',')
    except:
        # If URL fails, try loading from local file
        print("URL load failed. Attempting to load from local file...")
        try:
            df = pd.read_csv("./Data.csv", sep=',')
        except FileNotFoundError:
            raise FileNotFoundError("Could not load data from URL or local file './Train.csv'")

    # Rest of the function remains unchanged
    features = list(df)
    del features[0]
    del features[0]
    del features[len(features)-1]

    data = df.to_numpy()
    data = np.delete(data,1,axis=1)
    data = data.astype('float64')

    index = data[:,0]
    X = data[:,1:-1]
    y = data[:,-1]

    return X, y, features, index

# The Predictive Problem




### Description

Bike-sharing systems provide a convenient means of renting bicycles, with automated processes for membership, rental, and returns facilitated through a network of kiosks distributed across a city. These systems allow users to rent a bike from one location and return it to another, as needed. Globally, there are over 500 bike-sharing programs.

The data collected by these systems is highly valuable for researchers. Information such as travel duration, departure and arrival locations, and elapsed time is explicitly recorded, making these systems function as a dynamic sensor network for studying urban mobility. In this competition, participants are tasked with leveraging historical usage patterns and weather data to predict bike rental demand for the Capital Bikeshare program in Washington, D.C.

---

### Predictive Task

You are provided with hourly bike rental data spanning two years. The training set includes data from the first 19 days of each month, while the test set covers data from the 20th day to the end of the month. Your objective is to predict whether bike rental demand during each hour in the test set was **high** or **low**, using only information available prior to the rental period.

---

### Data Fields

- **yr**: Year (0 for 2011, 1 for 2012).
- **month**: Month of the year (1-12).
- **day**: Day of the month (1-31).
- **hour**: Hour of the day (0-23).
- **season**: Season of the year:
  - 1 = Spring
  - 2 = Summer
  - 3 = Fall
  - 4 = Winter
- **holiday**: Indicator of whether the day is a holiday (1 = holiday, 0 = non-holiday).
- **weekday**: Day of the week (0-6)
- **workingday**: Indicator of whether the day is a working day (1 = working day, 0 = weekend or holiday).
- **weather**: Weather conditions categorized as:
  - 1: Clear, Few clouds, Partly cloudy
  - 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds
  - 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds
  - 4: Heavy Rain + Ice Pellets + Thunderstorm + Mist, Snow + Fog
- **temp**: Actual temperature (normalized value).
- **atemp**: Perceived "feels-like" temperature (normalized value).
- **humidity**: Relative humidity percentage ((normalized value).
- **windspeed**: Wind speed (normalized value).
- **demand**: Target variable indicating rental demand:
  - 0 (or False): Low demand
  - 1 (or True): High demand


# 1. Data Exploration and Preprocessing


This code loads the bike sharing training dataset into features (X_train), target values (y_train), feature names (X_features), and index values (index) using the load_data() function.

In [ ]:
# load the dataset
X_train, y_train, X_features, index = load_data()

# data is stored in numpy array/matrix
print(f"X Shape: {X_train.shape}, X Type:{type(X_train)})")
print(f"y Shape: {y_train.shape}, y Type:{type(y_train)})")


- The following visualization is organized into **12 subplots** arranged in a **3x4 grid**, with each subplot representing the relationship between a specific feature and bike rental demand.

- Each bar in the subplots illustrates the **percentage distribution** between:
  - **High demand** (green)
  - **Low demand** (red)

- For **numeric features** (e.g., temperature, feels-like temperature, humidity, and windspeed):
  - The data is **discretized into 4 bins** to improve clarity and make the visualization more interpretable.

- This approach allows for an easy comparison of how different feature values influence rental demand.


In [ ]:
# Create subplots for each feature
fig, ax = plt.subplots(3, 4, figsize=(20, 15))
fig.suptitle('High vs Low Demand Distribution by Features', fontsize=20)

# Flatten ax array for easier iteration
ax = ax.ravel()

for i in range(len(ax)):
    feature_vals = X_train[:, i]

    # For numeric features (temp, atemp, humidity, windspeed), create bins
    if X_features[i] in ['temp', 'atemp', 'humidity', 'windspeed']:
        # Create 4 bins
        bins = np.linspace(feature_vals.min(), feature_vals.max(), 5)
        labels = [f'{bins[j]:.1f}-{bins[j+1]:.1f}' for j in range(len(bins)-1)]
        feature_vals = np.digitize(feature_vals, bins[1:-1])
        unique_vals = np.arange(len(labels))
        x_labels = labels
    else:
        # For categorical features, use as-is
        unique_vals = np.unique(feature_vals)
        x_labels = [str(int(val)) for val in unique_vals]

    high_demand = []
    low_demand = []

    # Calculate counts for high and low demand
    for val in unique_vals:
        mask = feature_vals == val
        total = np.sum(mask)
        if total > 0:
            high_count = np.sum(y_train[mask] == 1)
            low_count = np.sum(y_train[mask] == 0)
            # Convert to percentages
            high_demand.append((high_count / total) * 100)
            low_demand.append((low_count / total) * 100)

    # Create stacked bar plot
    ax[i].bar(x_labels, low_demand, label='Low Demand', color='red', alpha=0.7)
    ax[i].bar(x_labels, high_demand, bottom=low_demand, label='High Demand', color='green', alpha=0.7)

    ax[i].set_xlabel(X_features[i], fontsize=14)
    ax[i].set_ylabel('Percentage (%)', fontsize=14)
    ax[i].tick_params(axis='x', rotation=45 if len(x_labels) > 4 else 0)
    ax[i].grid(True, linestyle='--', alpha=0.3)
    ax[i].tick_params(axis='both', which='major', labelsize=12)
    ax[i].legend()

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

## TASK 1: Data Analysis (15 min)

Looking at the figures, choose what you think is the most and least informative feature to predict bike demand. Justify your answer.

Introduce your answer in the cell below.




### Answer (approx 200 words)
[Introduce your answer here]





# 2. Model Defintion



The provided code snippet demonstrates how to build, train, and evaluate a **logistic regression model** using Keras, a popular deep learning library. The model is designed to predict high demand in a bike-sharing dataset.



## TASK 2A: HYPERPARAMETER TUNING (30 min)


When training your logistic regression model using Keras, selecting the right hyperparameters is essential to ensure optimal performance and generalization to unseen data. In this task, you are required to determine suitable values for three key hyperparameters: the **learning rate**, the **number of epochs**, and the **validation split**. Each of these decisions plays a critical role in shaping the training process and the model's final accuracy.

For each hyperparameter, provide a clear rationale for your chosen values. To guide your reasoning, consider the following:

- **Learning Rate**:
  How does your selected learning rate balance the trade-off between model convergence speed and training stability? What potential issues, such as slow convergence or overshooting, could arise with suboptimal values?

- **Number of Epochs**:
  Why is your chosen number of epochs appropriate, given the complexity of your dataset and the risk of overfitting or underfitting? How do you plan to monitor performance during training to refine this choice?

- **Validation Split**:
  How does your chosen validation split ensure a robust and reliable evaluation of your model’s performance during training? What considerations (e.g., dataset size or class balance) influenced this decision?

Provide your reasoning and selected values in the cell below. Be sure to connect your choices to the specifics of your dataset and model to demonstrate a thoughtful approach to hyperparameter tuning.

**If the training takes longer than 2 min consider stop the training and do not explore these configurations.**


In [ ]:
# Import necessary libraries from Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# Define the model
model = Sequential()
model.add(Dense(1, input_dim=X_train.shape[1], activation='sigmoid'))  # Logistic regression

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model and store history
history = model.fit(X_train, y_train,
                   epochs=1,
                   batch_size=32,
                   validation_split=0.01,
                   verbose=1)

# Plot training history
plt.figure(figsize=(12, 4))

# Plot loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

# Print final metrics
print(f"Final Training Loss: {history.history['loss'][-1]:.4f}")
print(f"Final Training Accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"Final Validation Loss: {history.history['val_loss'][-1]:.4f}")
print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}")

### Answer (approx 300 words)
[Introduce your answer here]






## TASK 2B: Regularization (15 min)

Regularization is a key technique in machine learning that prevents overfitting by adding a penalty to the loss function. Here you will explore how regularization affects the performance of a logistic regression model on the bike-sharing dataset.

1. Train a logistic regression models using Keras and L2 regularization.
* Use the Dense layer in Keras with the `kernel_regularizer` parameter to apply the regularization.
* Experiment with different values of the regularization strength and note their impact.

2. Evaluate the Models:

* Compare the performance of both models using accuracy and validation loss.

In [ ]:
# Import necessary libraries from Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import regularizers

# Define the model
model = Sequential()
model.add(Dense(1, input_dim=X_train.shape[1], kernel_regularizer=regularizers.L2(0.0), activation='sigmoid'))  # Logistic regression

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model and store history
history = model.fit(X_train, y_train,
                   epochs=1,
                   batch_size=32,
                   validation_split=0.01,
                   verbose=1)

# Plot training history
plt.figure(figsize=(12, 4))

# Plot loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

# Print final metrics
print(f"Final Training Loss: {history.history['loss'][-1]:.4f}")
print(f"Final Training Accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"Final Validation Loss: {history.history['val_loss'][-1]:.4f}")
print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}")

### Answer
 1. Briefly explain in the purpose of regularization and how L2 regularization works. (150 words)

**[Introduce your answer here]**


 2. Discuss how the regularization terms affected the models in terms of overfitting and underfitting. (200 words)

**[Introduce your answer here]**


## TASK 2C: CREATING A MULTI-LAYER PERCEPTRON (30 min)

Now that you have gained experience training models, your next task is to create a **multilayer perceptron (MLP)** using Keras and train it as you did in previous exercises. Follow the steps below to guide your experimentation:

- **Create and Experiment with Different Architectures**:
  - Vary the **number of layers**.
  - Adjust the **number of neurons in each layer (width)** .
  - Use **different activation functions**.

-  **Train the MLP with L2 regularization**.
   - Use the Dense layer in Keras with the kernel_regularizer parameter to apply the regularization.
   - Experiment with different values of the regularization strength and note their impact.

- **Train and Evaluate**:
  - Train each configuration and monitor the key metrics (e.g., accuracy, loss) to compare different configurations.

- **Analyze and Discuss**:
  - Discuss your findings and justify which one would be the best configuration for this dataset and problem.


**If the training takes longer than 2 min consider stop the training and do not explore these configurations.**


In [ ]:
#Introduce the code for the MLP here
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# Define the model
model = Sequential()
model.add(Dense(1, input_dim=X_train.shape[1], activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model and store history
history = model.fit(X_train, y_train,
                   epochs=1,
                   batch_size=32,
                   validation_split=0.01,
                   verbose=1)

# Plot training history
plt.figure(figsize=(12, 4))

# Plot loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()
# Print final metrics
print(f"Final Training Loss: {history.history['loss'][-1]:.4f}")
print(f"Final Training Accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"Final Validation Loss: {history.history['val_loss'][-1]:.4f}")
print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}")

### Answer (300 words)

[Introduce your answer here]




# Task 3. Discussion of Key Take-aways (30 min)

In this section, you are expected to articulate the most relevant insights gained throughout the workflow. Use precise terminology consistent with the course material and ensure that your discussion explicitly connects to concepts introduced in the lectures and exercises.

## 3a. Dataset Take-aways (approx. 100 words)
[Insert your reflection on data quality, feature distributions, preprocessing steps, potential biases, missingness patterns, and any domain-specific considerations that influenced modelling decisions.]

## 3b. Model Take-aways (approx. 100 words)
[Insert your discussion of model selection rationale, inductive biases, hyperparameter settings, training dynamics, strengths and limitations of the chosen architecture, and how these relate to theoretical concepts covered in the course.]

## 3c. Evaluation Take-aways (approx. 100 words)
[Insert your analysis of evaluation metrics, validation strategy, overfitting/underfitting diagnosis, error patterns, model robustness, and how the results inform the model’s expected generalization performance.]

